In [4]:
import pandas as pd 
import numpy as np
data = pd.read_csv("train.csv") 
data.dtypes


satisfaction_level       float64
last_evaluation          float64
number_project             int64
average_montly_hours       int64
time_spend_company         int64
Work_accident              int64
left                       int64
promotion_last_5years      int64
sales                     object
salary                    object
dtype: object

In [6]:
temp5 = list(data.index.values)
numberOfPlus = 0
numberOfNeg = 0
for i in temp5:
    if str(data.at[i,"left"])==str(1):
        numberOfPlus += 1
    else:
        numberOfNeg += 1
print(numberOfPlus)
print(numberOfNeg)
ProPos = numberOfPlus/(numberOfPlus+numberOfNeg)
ProNeg = numberOfNeg/(numberOfPlus+numberOfNeg)
baseLineAccuracy = (ProPos*ProPos)+(ProNeg*ProNeg)
print(ProPos)
print(baseLineAccuracy)

2675
8563
0.2380316782345613
0.637254803217201


In [7]:
ran = np.random.rand(len(data)) < 0.8
train = data[ran]
val = data[~ran]
print("length of training data is "+str(len(train)))
print("length of validation data is "+str(len(val)))
temp5 = list(val.index.values)
numberOfPlus = 0
numberOfNeg = 0

for i in temp5:
    if str(val.at[i,"left"])==str(1):
        numberOfPlus += 1
    else:
        numberOfNeg += 1
print(type(i))
print(numberOfPlus)
print(numberOfNeg)
print(numberOfPlus/(numberOfNeg+numberOfPlus))

length of training data is 9056
length of validation data is 2182
<class 'numpy.int64'>
512
1670
0.23464711274060496


In [6]:
#categorical data or numerical data
#train = train.loc[:,['Work_accident', 'promotion_last_5years', 'sales', 'salary','left'] ] 
#val = val.loc[:,['Work_accident', 'promotion_last_5years', 'sales', 'salary','left'] ] 


In [8]:
class Node(object):
    def __init__(self, examples, attributes, decisionAttribute, decisionAttributeType, decisionAttributeValue, nodeNumber, parentDecisionAttribute, parentDecisionAttributeValue, decision, level):
        self.examples = examples
        self.attributes = attributes
        self.decisionAttribute = decisionAttribute 
        self.decisionAttributeType = decisionAttributeType
        self.decisionAttributeValue = decisionAttributeValue
        self.nodeNumber = nodeNumber
        self.children = []
        self.parentDecisionAttribute = parentDecisionAttribute
        self.parentDecisionAttributeValue = parentDecisionAttributeValue
        self.decision = decision
        self.level = level
    def add_child(self, obj):
        self.children.append(obj)
        
    def increment_nodeNumber(self):
        #nodeNumber = nodeNumber+1
        self.nodeNumber = self.nodeNumber + 1
        
    def __str__(self):
        #return self.label +"  "+ self.name+"  "+self.parentAttrValue+"Node number is "+ str(self.nodeNumber)+" of level "+str(self.level)
        return "Decision attribute for this node is "+self.decisionAttribute+" decision "+ self.decision+" Node number is "+ str(self.nodeNumber)+" of level "+str(self.level)+" number of examples of this node "+str(len(self.examples))

In [9]:
targetAttribute = 'left'
Attributes = list(train.columns)
Attributes.remove(targetAttribute)
AllAttrExpTar = Attributes[:]
print(Attributes)
examples = list(train.index.values) 
len(examples)

['satisfaction_level', 'last_evaluation', 'number_project', 'average_montly_hours', 'time_spend_company', 'Work_accident', 'promotion_last_5years', 'sales', 'salary']


9056

In [10]:
def getEntropyOfExamples(examplesForCurAttr):
    NumberOfPosEx = 0
    NumberOfNegEx = 0
    for j in examplesForCurAttr:
        if  str(train.at[int(j),targetAttribute]) == str(1):
            NumberOfPosEx = NumberOfPosEx+1
        elif str(train.at[int(j),targetAttribute]) == str(0):
             NumberOfNegEx = NumberOfNegEx+1
    PropOfPos =  NumberOfPosEx/len(examplesForCurAttr)
    PropOfNeg = NumberOfNegEx/len(examplesForCurAttr)
    if PropOfPos == 0:
        return (-1*PropOfNeg*(np.log(PropOfNeg)/np.log(2)))
    elif PropOfNeg == 0:
        return (-1*PropOfPos*(np.log(PropOfPos)/np.log(2)))
    else:
        return ((-1*PropOfPos*(np.log(PropOfPos)/np.log(2))) + (-1*PropOfNeg*(np.log(PropOfNeg)/np.log(2))))

In [87]:
def getMethodImpurity(numericalAttribute,value,examplesForCurAttr,method):
    #print("calculating Impurity")
    
    fra1 = train.loc[examplesForCurAttr,:]
    LessThanExamples = []
    MoreThanExamples = []
    for j in examplesForCurAttr:
        if  float(str(train.at[int(j),numericalAttribute])) <= value:
            LessThanExamples.append(int(j))
        else:
            MoreThanExamples.append(int(j))
    if len(LessThanExamples) ==0 or len(MoreThanExamples) == 0:    
        print("I am here and there is a problem")
    if len(LessThanExamples) ==0 or len(MoreThanExamples) == 0:
        return 999
    else:
        if getEntropyOfExamples(LessThanExamples) ==0 or getEntropyOfExamples(MoreThanExamples) == 0:
            print("yes i am here kartheek")        
        return ((len(LessThanExamples)*getEntropyOfExamples(LessThanExamples))+(len(MoreThanExamples)*getEntropyOfExamples(MoreThanExamples)))/len(examplesForCurAttr)
    


In [88]:
NumericalAttributes = ['satisfaction_level', 'last_evaluation', 'number_project', 'average_montly_hours', 'time_spend_company']
CategoricalAttributes = ['Work_accident', 'promotion_last_5years', 'sales', 'salary']

def getImpurity(currentAttribute,examplesForCurAttr,targetAttribute,method):
    
    NumberOfexamplesForCurAttr = len(examplesForCurAttr)
    sum = 0.0
    fra1 = train.loc[examplesForCurAttr,[currentAttribute] ]
    currentAttributeValues = fra1[currentAttribute].unique()
    for i in currentAttributeValues:
        NumberOfExaForThisValue = 0
        for j in examplesForCurAttr:
            if str(train.at[int(j),currentAttribute]) == str(i):
                NumberOfExaForThisValue = NumberOfExaForThisValue+1
        if NumberOfExaForThisValue == 0:
            sum = sum+0
            print("yup I am here")
        else:
            NumberOfPosForThisValue = 0
            NumberOfNegForThisValue = 0
            for j in examplesForCurAttr:
                if str(train.at[int(j),targetAttribute]) == str(1) and str(train.at[int(j),currentAttribute]) == str(i):
                    NumberOfPosForThisValue = NumberOfPosForThisValue+1
                elif str(train.at[int(j),targetAttribute]) == str(0) and str(train.at[int(j),currentAttribute]) == str(i):
                    NumberOfNegForThisValue = NumberOfNegForThisValue+1
            PropOfPos = NumberOfPosForThisValue/NumberOfExaForThisValue
            PropOfNeg = NumberOfNegForThisValue/NumberOfExaForThisValue
            PropOfThisValue = NumberOfExaForThisValue/NumberOfexamplesForCurAttr            
            if method == 'Gini':
                sum = sum + (PropOfThisValue*2*PropOfPos*PropOfNeg)
            elif method == 'misClasifi':
                sum = sum + (PropOfThisValue*min(PropOfPos,PropOfNeg))
            elif method == 'Entropy':
                if PropOfPos == 0:
                    sum = sum + ((PropOfThisValue)*(-1*PropOfNeg*(np.log(PropOfNeg)/np.log(2))))
                elif PropOfNeg == 0:
                    sum = sum + ((PropOfThisValue)*(-1*PropOfPos*(np.log(PropOfPos)/np.log(2))))
                else:
                    sum = sum + ((PropOfThisValue)*((-1*PropOfPos*(np.log(PropOfPos)/np.log(2))) + (-1*PropOfNeg*(np.log(PropOfNeg)/np.log(2)))))
    return sum

def BestOfAttributes(Attributes,examplesForCurAttr,targetAttribute,method):
    entropyForCurrentExamples = getEntropyOfExamples(examplesForCurAttr)
    EntropyForCategoricalAttributes = {}
    for i in CategoricalAttributes:
        EntropyForCategoricalAttributes[i] = getImpurity(i,examplesForCurAttr,targetAttribute,method)
    BestOfCategoricalAttributes = min(EntropyForCategoricalAttributes, key=EntropyForCategoricalAttributes.get)
    BestOfCategoricalAttributesValue = min(list(EntropyForCategoricalAttributes.values()))            
    fra1 = train.loc[examplesForCurAttr,NumericalAttributes]
    #fra2 = fra1[:]
    #print(fra1)
    dictForNumAttrUniqueValues = {}
    for i in NumericalAttributes:
        dictForNumAttrUniqueValues[i] = list(fra1[i].sort_values().unique())
    dictForEntropyUniqueValue = dictForNumAttrUniqueValues.copy()
    print(dictForNumAttrUniqueValues)
    for i in NumericalAttributes:
        for j,val in enumerate(dictForNumAttrUniqueValues[i]):
            dictForEntropyUniqueValue[i][j] = getMethodImpurity(i,dictForNumAttrUniqueValues[i][j],examplesForCurAttr,method)          
    EntropyValuesForNumericalAttributes = {}
    for i in NumericalAttributes:
        EntropyValuesForNumericalAttributes[i]=min(dictForEntropyUniqueValue[i])
    BestOfNumericalAttributes = min(EntropyValuesForNumericalAttributes, key=EntropyValuesForNumericalAttributes.get)
    BestOfNumericalAttributesValue = min(EntropyValuesForNumericalAttributes.values())
    for i in dictForNumAttrUniqueValues[BestOfNumericalAttributes]:
        if i == BestOfNumericalAttributesValue:
            break
    BestSplitValue = i       
    if len(Attributes) > 5:
        if BestOfNumericalAttributesValue < BestOfCategoricalAttributesValue:
            return BestOfNumericalAttributes,BestSplitValue
        else: 
            return BestOfCategoricalAttributes,0
    else:
        return BestOfNumericalAttributes,BestSplitValue    

#(self, examples, attributes, decisionAttribute, decisionAttributeType, decisionAttributeValue, nodeNumber, parentDecisionAttribute, parentDecisionAttributeValue, decision, level):
def DecisionTreeBuilder(examplesForCurNode,targetAttribute,AttributesForCurrentNode,parentDecisionAttribute,parentAttrValue,level):
    global numberOfNodes
    allExPos = True
    for i in examplesForCurNode:
        if str(train.at[int(i),targetAttribute]) == str(1):
            pass
        else:
            allExPos = False
            break
    if allExPos:        
        r = Node(examplesForCurNode,AttributesForCurrentNode,'leafNode','leafNode','leafNode',numberOfNodes,parentDecisionAttribute,parentAttrValue,'Positive',level)
        print("Type1")
        return r
    allExNeg = True
    for j in examplesForCurNode:
        if str(train.at[j,targetAttribute]) == str(0):
            pass
        else:
            allExNeg = False
            break
    if allExNeg:
        r = Node(examplesForCurNode,AttributesForCurrentNode,'leafNode','leafNode','leafNode',numberOfNodes,parentDecisionAttribute,parentAttrValue,'Negative',level)
        print("Type2")
        return r
    if len(AttributesForCurrentNode) == 0 or getEntropyOfExamples(examplesForCurNode) < 0.01 or len(examplesForCurNode)<50:
        fra1 = train.loc[examplesForCurNode,[targetAttribute] ]
        targetAttributesList = fra1[targetAttribute].values
        unique1, counts1 = np.unique(targetAttributesList, return_counts=True)
        dictOfTaAttr = dict(zip(unique1, counts1))
        posCount = dictOfTaAttr[1]
        negCount = dictOfTaAttr[0]
        if (posCount/(posCount+negCount))>0.5:
            r = Node(examplesForCurNode,AttributesForCurrentNode,'leafNode','leafNode','leafNode',numberOfNodes,parentDecisionAttribute,parentAttrValue,'Positive',level)
            numberOfNodes  = numberOfNodes + 1
        else:
            r = Node(examplesForCurNode,AttributesForCurrentNode,'leafNode','leafNode','leafNode',numberOfNodes,parentDecisionAttribute,parentAttrValue,'Negative',level)
            numberOfNodes  = numberOfNodes + 1
        print("Type3")
        return r
    BestAttribute,BestAttributeValue = BestOfAttributes(AttributesForCurrentNode,examplesForCurNode,targetAttribute,'misClasifi')
    if BestAttribute in NumericalAttributes:
        TypeOfdecisionAttribute = 'numerical'
    else:
        TypeOfdecisionAttribute = 'categorical'
        
    if TypeOfdecisionAttribute == 'categorical':
        r = Node(examplesForCurNode,AttributesForCurrentNode,BestAttribute,TypeOfdecisionAttribute,BestAttributeValue,numberOfNodes,parentDecisionAttribute,parentAttrValue,'NotLeaf',level)
        print("Type4")
        numberOfNodes  = numberOfNodes + 1
        BestAttributeValues = list(data[BestAttribute].unique())
        for iter in BestAttributeValues:
            AttributesForChildren = []
            AttributesForChildren = AttributesForCurrentNode[:]
            AttributesForChildren.remove(BestAttribute)
            #AttributesForChildren = AttributesForChildren + NumericalAttributes
            examplesForThisChildOfCurrentNode = []
            for iter1 in examplesForCurNode:
                if  str(train.at[int(iter1),str(BestAttribute)]) == str(iter):
                    examplesForThisChildOfCurrentNode.append(int(iter1))
            print("examplesForThisChildOfCurrentNode is "+str(len(examplesForThisChildOfCurrentNode)))
            if len(examplesForThisChildOfCurrentNode)==0:
                numberOfNodes  = numberOfNodes + 1
                fra1 = train.loc[examplesForCurNode,[targetAttribute] ]
                targetAttributesList = fra1[targetAttribute].values
                unique1, counts1 = np.unique(targetAttributesList, return_counts=True)
                dictOfTaAttr = dict(zip(unique1, counts1))
                posCount = dictOfTaAttr[1]
                negCount = dictOfTaAttr[0]
                if (posCount/(posCount+negCount))>0.5:
                    r.add_child(Node(examplesForThisChildOfCurrentNode,AttributesForCurrentNode,'leafNode','leafNode','leafNode',numberOfNodes,BestAttribute,str(iter),'Positive',level+1))                
                    print("Type5")
                else:
                    r.add_child(Node(examplesForThisChildOfCurrentNode,AttributesForCurrentNode,'leafNode','leafNode','leafNode',numberOfNodes,BestAttribute,str(iter),'Negative',level+1))                    
                    print("Type6")
                numberOfNodes = numberOfNodes - 1
            else:        
                r.add_child(DecisionTreeBuilder(examplesForThisChildOfCurrentNode,targetAttribute,AttributesForChildren,BestAttribute,str(iter),level+1))
        print("********************************************************************")
        print("The label of current node is")
        print(r)
        #print("the best decision attribute of current node is")
        #print(BestAttribute)
        #print("number of examplesForCurNode of this attribute is ")
        #print(len(examplesForCurNode))
        #print(examplesForCurNode)
        #print("children of current nodes")
        plabo = range(0,len(r.children))
        for k in plabo:
            pass
            #print(r.children[k])
        print("Type7")
        print("********************************************************************")

        return r
    else:
        print(BestAttribute)
        print(BestAttributeValue)
        r = Node(examplesForCurNode,AttributesForCurrentNode,BestAttribute,TypeOfdecisionAttribute,BestAttributeValue,numberOfNodes,parentDecisionAttribute,parentAttrValue,'NotLeaf',level)
        print("Type8")
        numberOfNodes  = numberOfNodes + 1
        #TheBestAttributeValues = [str(less than)+str(BestAttributeValue),str(greater than)+str(BestAttributeValue)]
        #for iter in TheBestAttributeValues:
        AttributesForChildren = []
        AttributesForChildren = AttributesForCurrentNode[:]
        examplesForLeftChildOfCurrentNode = []
        examplesForRightChildOfCurrentNode = []
        for iter1 in examplesForCurNode:
            if  float(str(train.at[int(iter1),str(BestAttribute)])) <= float(BestAttributeValue):
                examplesForLeftChildOfCurrentNode.append(int(iter1))
            else:
                examplesForRightChildOfCurrentNode.append(int(iter1))
        r.add_child(DecisionTreeBuilder(examplesForLeftChildOfCurrentNode,targetAttribute,AttributesForChildren,BestAttribute,BestAttributeValue,level+1))
        r.add_child(DecisionTreeBuilder(examplesForRightChildOfCurrentNode,targetAttribute,AttributesForChildren,BestAttribute,BestAttributeValue,level+1))
        return r
    return r
                
#decisionAttribute == 'leafNode' if leaf or bestdecision at internal node
#decisionAttributeType == 'categorical' or 'numerical' or 'leafNode' if leaf
#decisionAttributeValue = 'leafNode' if leaf
    
def predict(i,r,TestSet):
    if r.decisionAttribute == 'leafNode':
        if r.decision == 'Positive':
            return '1'
        else:
            #print("I am here in leaft")
            return '0'
    else:
        if r.decisionAttributeType == 'categorical': 
            AttrValue = str(TestSet.at[int(i),str(r.decisionAttribute)])
            #print(TestSet.at[int(i),str(r.decisionAttribute)])
            for j in r.children:
                if str(j.parentDecisionAttributeValue) == str(AttrValue):
                    break
            #print(j.parentDecisionAttributeValue)
            #print(AttrValue)
            if str(j.parentDecisionAttributeValue) == str(AttrValue):
                pass
            else:
                print("there is a problem")
            return predict(i,j,TestSet)
        elif r.decisionAttributeType == 'numerical':
            AttrValue = float(str(TestSet.at[int(i),str(r.decisionAttribute)]))
            if AttrValue > r.decisionAttributeValue:
                return predict(i,r.children[1],TestSet)
            else:
                return predict(i,r.children[0],TestSet)
                
            
            
            
        
    

In [89]:
numberOfNodes = 0
examplesForRootNode = examples[:]
AttributesForRootNode = Attributes[:]
RootNode = DecisionTreeBuilder(examplesForRootNode,targetAttribute,AttributesForRootNode,'root','root',0)
numberOfNodes

{'satisfaction_level': [0.09, 0.1, 0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2, 0.21, 0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3, 0.31, 0.32, 0.33, 0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4, 0.41, 0.42, 0.43, 0.44, 0.45, 0.46, 0.47, 0.48, 0.49, 0.5, 0.51, 0.52, 0.53, 0.54, 0.55, 0.56, 0.57, 0.58, 0.59, 0.6, 0.61, 0.62, 0.63, 0.64, 0.65, 0.66, 0.67, 0.68, 0.69, 0.7, 0.71, 0.72, 0.73, 0.74, 0.75, 0.76, 0.77, 0.78, 0.79, 0.8, 0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87, 0.88, 0.89, 0.9, 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.99, 1.0], 'last_evaluation': [0.36, 0.37, 0.38, 0.39, 0.4, 0.41, 0.42, 0.43, 0.44, 0.45, 0.46, 0.47, 0.48, 0.49, 0.5, 0.51, 0.52, 0.53, 0.54, 0.55, 0.56, 0.57, 0.58, 0.59, 0.6, 0.61, 0.62, 0.63, 0.64, 0.65, 0.66, 0.67, 0.68, 0.69, 0.7, 0.71, 0.72, 0.73, 0.74, 0.75, 0.76, 0.77, 0.78, 0.79, 0.8, 0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87, 0.88, 0.89, 0.9, 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.99, 1.0], 'number_project': [2, 3, 4, 5

I am here and there is a problem
Type4
examplesForThisChildOfCurrentNode is 2171
{'satisfaction_level': [0.09, 0.1, 0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2, 0.21, 0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3, 0.31, 0.32, 0.33, 0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4, 0.41, 0.42, 0.43, 0.44, 0.45, 0.46, 0.47, 0.48, 0.49, 0.5, 0.51, 0.52, 0.53, 0.54, 0.55, 0.56, 0.57, 0.58, 0.59, 0.6, 0.61, 0.62, 0.63, 0.64, 0.65, 0.66, 0.67, 0.68, 0.69, 0.7, 0.71, 0.72, 0.73, 0.74, 0.75, 0.76, 0.77, 0.78, 0.79, 0.8, 0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87, 0.88, 0.89, 0.9, 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.99, 1.0], 'last_evaluation': [0.36, 0.37, 0.38, 0.39, 0.4, 0.41, 0.42, 0.43, 0.44, 0.45, 0.46, 0.47, 0.48, 0.49, 0.5, 0.51, 0.52, 0.53, 0.54, 0.55, 0.56, 0.57, 0.58, 0.59, 0.6, 0.61, 0.62, 0.63, 0.64, 0.65, 0.66, 0.67, 0.68, 0.69, 0.7, 0.71, 0.72, 0.73, 0.74, 0.75, 0.76, 0.77, 0.78, 0.79, 0.8, 0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87, 0.88, 0.89, 0.9, 0.91, 0.

ValueError: list.remove(x): x not in list

In [92]:
bolo = {'kartheek':2,'sravani':3}
key_min = min(bolo.values())
key_min
boro = {}
#key_min = min(boro.values())
key_min
[999]*5

[999, 999, 999, 999, 999]

In [21]:
temp1 = []
temp2 = [1,2,3]
temp2

[1, 2, 3]

In [ ]:
for i in NumericalAttributes:   
        temp = []
        temp = temp + list(fra1[i].unique())
        #temp = np.array(temp)
        #temp = temp.sort()
        #temp = list(temp.sort())
        print(temp)
        print(type(temp))
        temp1 = ([999]*(len(fra1[i])-len(temp)))
        fra2[i] = temp + temp1
        fra2[i] = fra2[i].sort_values()
        print(fra2[i])
    #print(fra2)   
    for i in NumericalAttributes:
        for j in list(fra2.index.values):
            if (fra2.at[int(j),i]) == 0:
                print("here it is zero")
                print(i)
                print(j)
                fra2.at[int(j),i] == 999
                
    fra3 = fra2[:]
    #fra2[:] = 999
    for i in NumericalAttributes:
        for j in list(fra2.index.values):
            if fra2.at[int(j),i] != 999:
                fra3.at[int(j),i] = getMethodImpurity(i,fra2.at[int(j),i],examplesForCurAttr,method)
                if (fra3.at[int(j),i]) == 0:
                    print("the attribute is ")
                    print(i)
                    print(fra2.at[int(j),i])
                    
                print(fra3.at[int(j),i])
            else:
                break
    EntropyValuesForNumericalAttributes = {}
    for i in NumericalAttributes:
        EntropyValuesForNumericalAttributes[i]=min(fra3[i])
    BestOfNumericalAttributes = min(EntropyValuesForNumericalAttributes, key=EntropyValuesForNumericalAttributes.get)
    BestOfNumericalAttributesValue = min(EntropyValuesForNumericalAttributes.values())
    for i in list(fra2.index.values):
        if fra3.at[int(i),BestOfNumericalAttributes] == BestOfNumericalAttributesValue:
            break
    BestSplitValue = fra2.at[int(i),BestOfNumericalAttributes]

In [91]:
print("hello")

hello
